# LDA

We sugesst to keep this notebook and work on a copy of this file that you can refer to this notebook whenever is necessary.


# 1- First import data and become familiar with the data. 

*To do so we should:* 

- Import required library 

- Import data set and become comftable with data 

In [4]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# 2-Some Data explanation is in order here **

In [5]:
categories = ['alt.atheism',
              'talk.religion.misc',
              'comp.graphics',
              'sci.space']

num_classes = len(categories)

In [6]:
train=fetch_20newsgroups(subset='train', categories=categories,shuffle=True)
test=fetch_20newsgroups(subset='test', categories=categories,shuffle=True)

No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


In [7]:
train_data=dict()
test_data=dict()

train_data['target'] = train.target
test_data['target'] = test.target

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,stop_words='english')
train_data['data'] = vectorizer.fit_transform(train.data)
test_data['data'] = vectorizer.transform(test.data)

with open('train.pkl','wb') as f0:
    pickle.dump(train_data,f0)
f0.close()
with open('test.pkl','wb') as f0:
    pickle.dump(test_data,f0)
f0.close()


# 3-Data Dimension

In [8]:
train_data['data'].shape

(2034, 33809)

In [9]:
feature_dim = train_data['data'].shape[1]
feature_dim

33809

In [23]:
item_index = np.where(train_data['target']==0)
item_index[0]

array([   3,    5,   16,   30,   34,   36,   38,   40,   42,   45,   46,
         55,   61,   67,   76,   77,   82,   86,   88,   92,   94,   96,
        100,  101,  108,  119,  120,  125,  128,  137,  142,  145,  146,
        149,  150,  169,  175,  176,  182,  189,  190,  192,  196,  197,
        202,  208,  210,  212,  213,  218,  219,  221,  225,  229,  231,
        233,  242,  243,  245,  257,  261,  266,  272,  278,  279,  282,
        287,  294,  296,  306,  307,  310,  315,  316,  322,  324,  327,
        331,  332,  334,  346,  350,  362,  364,  367,  368,  370,  372,
        377,  379,  380,  386,  392,  395,  398,  399,  409,  410,  411,
        412,  419,  422,  424,  427,  440,  443,  448,  453,  455,  462,
        467,  469,  477,  478,  493,  510,  511,  519,  520,  521,  527,
        529,  530,  533,  536,  540,  543,  544,  545,  550,  551,  554,
        561,  564,  582,  586,  589,  590,  591,  602,  609,  612,  617,
        620,  624,  630,  631,  639,  645,  653,  6

# 4-LDA

*If you start by applying LDA directely on the training data, you will encounter memory crash since the number of feasures is very big. Therefore, a dimension reduction is necessary.*

- We use PCA to reduce the feasure dimension.

- For now set the reduction factor to .005 to reduce the runing time until you make sure your code works. Then set the reduction factor to .03 to see if that improves the model performance.

- Run the following cell to compute the shrunk data set. We use thePCA model built in Scikit-Learn library. 

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=int(.03*feature_dim))
train_data_shrunk = pca.fit(train_data['data'].todense()).transform(train_data['data'].todense())
test_data_shrunk = pca.transform(test_data['data'].todense())

deducted_feature_dim = train_data_shrunk.shape[1]
deducted_feature_dim

1014

*In following cell we are going to compute LDA parameters based on the formulas in (4.36), (4.37) and (4.38). The difference here is that we are not going to compute the denominator in (3.38) since that is not going to change the class score. Please make sure you have understood this. We should compute following parameters:*

- mu: The mean of feasures in each class. 

- Sigma: The covariance matrix of each class. Be aware that in LDA we assumed that the covariance of classes are the same. Therefore, in real application we take the average of the covariance of all classes.

- Pi: The class prior.

- Sigma_ave: The average of covariance matrices of all classes.

- beta: The parameter given in (4.37)

- gamma: The parameter given in (4.36)

In [75]:
def calculate_sigma_and_mu(train_data, train_data_shrunk, category):
    index = np.where(train_data['target'] == category)[0]
    vectors = train_data_shrunk[index]
    mean = vectors.mean(axis=0)
    covariance = np.cov(vectors, rowvar=False)
    return mean, covariance

In [81]:
_, y_t = np.unique(train_data['target'], return_inverse=True)
prior = np.bincount(y_t) / float(len(train_data['target']))

In [44]:
mu = np.zeros([train_data_shrunk.shape[1],len(categories)], dtype=float)
Sigma = np.zeros([train_data_shrunk.shape[1],train_data_shrunk.shape[1],len(categories)])
Pi = np.zeros(len(categories),dtype=float)

for i in range(0,len(categories)):
    category_sigma, category_mu = calculate_sigma_and_mu(train_data, train_data_shrunk, i)
    mu_matrix[:,category] = mean
    Sigma_matrix[:,:,category] = np.cov(vectors, rowvar=False)

Sigma_ave = Sigma.mean(axis=2)
beta = 
gamma = 


*In the following cell we are going to compute:*
- class_scores: The class score of each data point in the test data set. This is the numerator in (4.38) 
- class_prediction: The predicted class based on the class score which is going to be the maximum of all scores of classes.
- accuracy_rate: The rate of accurracy of our LDA implementation

In [46]:
class_scores = 
class_prediction = 
accuracy_rate = (**).mean()
accuracy_rate


0.71249076127124911

# 5-Compare with Other Linear Models

*In following cell we are going to compare LDA with other linear models built in SciKit-Learn*

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(train_data_shrunk,train_data['target']) 
(abs(neigh.predict(test_data_shrunk)-test_data['target'])!=0).mean()
